In [1]:
# This file contains Underwriting Staffing Optimization for an Insurance Company.
# Trinee Das and Anindya Bhaumik has worked on this Assignment together.

In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook

# Create a Dataframe for the Staff information
staff = pd.read_excel("Staffing+Data.xlsx", sheet_name ='StaffAvailability')
# Create a Dataframe for the Staffing Cost related information
cost = pd.read_excel("Staffing+Data.xlsx", sheet_name ='Cost')
# Create a Dataframe for the Service Rate related information
transcost = pd.read_excel("Staffing+Data.xlsx", sheet_name ='ServiceRate')
# Create a Dataframe for the Demand related information
mgdemandDist = pd.read_excel("Staffing+Data.xlsx", sheet_name ='DemandData')

## Data pre-processing 

In [4]:
# Create the required Python data structures for indexes and parameters

# Create a Dictionary object Demand with index: 'State' and 'Month' and Value as "Demand" column from the mgdemandDist Dataframe
Demand = mgdemandDist.set_index(['State','Month'])['Demand'].to_dict()
# Create a Dictionary object FTE_Salary with index: 'State' and 'Month' and Value as "MonthlySalary" column from the cost Dataframe
FTE_Salary = cost.set_index(['State','Month'])['MonthlySalary'].to_dict()
# Create a Dictionary object UnitOutSourceCost with index: 'State' and 'Month' and Value as "UnitOutSourceCost" column from the cost Dataframe
UnitOutSourceCost = cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

FTE_AppServRate = 40

StaffAvPer = staff.set_index(['State','Month'])['StaffAvPer'].to_dict()
StaffAv_LB = staff.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = staff.set_index(['State','Month'])['UB'].to_dict()

State = cost['State'].unique()
Month = cost['Month'].unique()

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [5]:
# Creating a model instance
model = ConcreteModel()

In [6]:
# Define Pyomo sets and Parameters
model.i = Set(initialize=State.tolist(),doc='States')
model.j = Set(initialize=Month.tolist(),doc='Months')
model.demand = Param(model.i,model.j,initialize = Demand,doc='Demand')
model.sa = Param(model.i,model.j,initialize = StaffAvPer,doc = 'StaffAvPercent')

In [7]:
# Decision variables
model.x = Var(model.i,model.j,doc='No of FTE',domain = NonNegativeReals)
model.y = Var(model.i,model.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [8]:
# Constraints

# Demand Constraint
def total_demand(m,i,j):
    return (m.x[i,j]*40*m.sa[i,j] + m.y[i,j] == m.demand[i,j])
            
model.total_demand = Constraint(model.i,model.j, rule=total_demand)

In [9]:
# Outsource Demand Constraints
model.outsource_demand = ConstraintList()
for j in model.j:
    for i in model.i:
        if i == 'A':
            model.outsource_demand.add(expr = model.y[i,j] <= 0.3*model.demand[i,j])
        elif i == 'B':
            model.outsource_demand.add(expr = model.y[i,j] <= 0.4*model.demand[i,j])

In [10]:
# Objective function
model.value = Objective(expr = sum(sum(model.x[i,j]*FTE_Salary[i,j] + model.y[i,j]*UnitOutSourceCost[i,j] for i in model.i) for j in model.j), sense= minimize)

In [11]:
# Invoking the solver
result = SolverFactory('glpk').solve(model)
result.write()

ERROR: Solver (glpk) returned non-zero return code (1)
ERROR: Solver log: GLPSOL: GLPK LP/MIP Solver, v4.65 Parameter(s) specified in
    the command line:
     --write C:\Users\trine\AppData\Local\Temp\tmpx8jfcfe5.glpk.raw --wglp
     C:\Users\trine\AppData\Local\Temp\tmpzkbm35tc.glpk.glp --cpxlp
     C:\Users\trine\AppData\Local\Temp\tmpd1x0h9nz.pyomo.lp
    Reading problem data from
    'C:\Users\trine\AppData\Local\Temp\tmpd1x0h9nz.pyomo.lp'...
    C:\Users\trine\AppData\Local\Temp\tmpd1x0h9nz.pyomo.lp:5: constraints
    section missing CPLEX LP file processing error


ApplicationError: Solver (glpk) did not exit normally

In [ ]:
# Print the value of the objective function
optimized_cost = round(model.value()/1000000,2)
optimized_cost

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [ ]:
model.pprint()

In [ ]:
# Creating dataframe for the results

#Creating an empty list
Output_actual = []


for i in model.i:
    for j in model.j:
        
        #cost for the application approval process
        no_of_fte=model.x[i,j].value
        demand = model.demand[i, j]
        no_of_outsource_app = model.y[i,j].value
        no_of_FTE_processed_app = model.x[i,j].value*40*model.sa[i,j]
        cost = model.x[i,j].value*FTE_Salary[i,j] + model.y[i,j].value*UnitOutSourceCost[i,j]
        percent_outsourced_app = round((no_of_outsource_app/demand)*100,2)
        avg_cost_per_app = round(cost/demand,2)
        
        Output_actual.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,avg_cost_per_app])
        
print(Output_actual)

In [ ]:
Output_actual = pd.DataFrame(Output_actual, columns = ['State', 'Month', 'No of FTE',"Demand",'No of outsource app',"No of FTE processed App" , 'Estimated Cost', 'percent outsourced app', 'average cost per application'])

Output_actual[['Estimated Cost in mn $']] = Output_actual[['Estimated Cost']]/1000000
Output_actual['Demand'] = Output_actual['Demand'].astype(int)
Output_actual['No of outsource app'] = Output_actual['No of outsource app'].astype(int)
Output_actual['No of FTE processed App'] = Output_actual['No of FTE processed App'].astype(int)
Output_actual = Output_actual.round({"No of FTE":1, "No of outsource app":0, "Estimated Cost in mn $":1})


In [ ]:
Output_actual

In [ ]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_actual'
Output_actual.to_excel(writer, sheet_name='Output_actual')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [ ]:
# Creating a model instance
model1 = ConcreteModel()


In [ ]:
# Define Pyomo sets and Parameters
model1.i = Set(initialize=State.tolist(),doc='States')
model1.j = Set(initialize=Month.tolist(),doc='Months')
model1.demand = Param(model1.i,model1.j,initialize = Demand,doc='Demand')
model1.sa = Param(model1.i,model1.j,initialize = StaffAv_LB,doc = 'StaffLowerBoundPercent')

In [ ]:
# Decision variables
model1.x = Var(model1.i,model1.j,doc='No of FTE',domain = NonNegativeReals)
model1.y = Var(model1.i,model1.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [ ]:
# Constraints
# Demand Constraint

def total_demand(m, i, j):
    return (m.x[i, j] * 40 * m.sa[i, j] + m.y[i, j] == m.demand[i, j])


model1.total_demand = Constraint(model1.i, model1.j, rule=total_demand)

# Outsource Demand Constraints
model1.outsource_demand = ConstraintList()
for j in model1.j:
    for i in model1.i:
        if i == 'A':
            model1.outsource_demand.add(expr = model1.y[i,j] <= 0.3*model1.demand[i,j])
        elif i == 'B':
            model1.outsource_demand.add(expr = model1.y[i,j] <= 0.4*model1.demand[i,j])

In [ ]:
# Objective function
model1.value = Objective(expr = sum(sum(model1.x[i,j]*FTE_Salary[i,j]+ model1.y[i,j]*UnitOutSourceCost[i,j] for i in model1.i) for j in model1.j),sense= minimize)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model1)
result.write()

In [ ]:
# Print the value of the objective function
optimized_cost_worst = round(model1.value()/1000000,2)

print("\n\n The company need to spend around ${}m in total for the application approval process" .format(optimized_cost_worst))




**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [ ]:
model1.pprint()

In [ ]:
# Creating dataframe for the results
Output_worst = []

for i in model1.i:
    for j in model1.j:
       
        no_of_fte = model1.x[i, j].value
        demand = model1.demand[i, j]
        no_of_outsource_app = model1.y[i, j].value
        no_of_FTE_processed_app = model1.x[i,j].value*40*model1.sa[i,j]
        cost = model1.x[i, j].value*FTE_Salary[i, j] + model1.y[i, j].value*UnitOutSourceCost[i, j]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        avg_cost_per_app = round(cost/demand,1)

        Output_worst.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,avg_cost_per_app])
print(Output_worst)        

In [ ]:
Output_worst = pd.DataFrame(Output_worst, columns = ['State', 'Month', 'No of FTE worst',"Demand",'No of outsource app worst',"No of FTE processed App worst" , 'Estimated Cost worst', 'percent outsourced app worst', 'average cost per application worst'])

Output_worst['Estimated Cost in mn $ worst'] = Output_worst[['Estimated Cost worst']]/1000000
Output_worst['Demand'] = Output_worst['Demand'].astype(int)
Output_worst['No of outsource app worst'] = Output_worst['No of outsource app worst'].astype(int)
Output_worst['No of FTE processed App worst'] = Output_worst['No of FTE processed App worst'].astype(int)
Output_worst = Output_worst.round({"No of FTE worst":1, "No of outsource app worst":0, "Estimated Cost in mn $ worst":1})
#Output_worst[['Cost per application worst']] = Output_worst['Estimated Cost worst']/Output_worst['Demand']
# Print the Output values
print("\n\n" ,Output_worst)

In [ ]:
total_outsourcing =  sum(Output_worst['No of outsource app worst'])
total_cost = sum(Output_worst['Estimated Cost worst'])
optimal_staff_members = round(Output_worst['No of FTE worst'].mean(),1)

total_demand =  sum(Output_worst['Demand'])
average_cost_per_appln = round(total_cost/total_demand,2)
percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)
print("Total Outsourced Application: ", total_outsourcing)
print("Total Demand Application: ", total_demand)
print("Optimal Staff Members : ", optimal_staff_members)

#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here
print("Total Outsourced Application % is : ", percent_outsourced_app_total)


#### 3.1.3 Average cost per application

In [ ]:
# write your code here
print("Average Cost per Application: ", average_cost_per_appln)

In [ ]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_worst'
Output_worst.to_excel(writer, sheet_name='Output_worst')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [ ]:
# Creating a model instance
model2 = ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
model2.i = Set(initialize=State.tolist(),doc='States')
model2.j = Set(initialize=Month.tolist(),doc='Months')
model2.demand = Param(model2.i,model2.j,initialize = Demand,doc='Demand')
model2.sa = Param(model2.i,model2.j,initialize = StaffAv_UB,doc = 'StaffUpperBoundPercent')

In [ ]:
# Decision variables
model2.x = Var(model2.i,model2.j,doc='No of FTE',domain = NonNegativeReals)
model2.y = Var(model2.i,model2.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [ ]:
# Constraints
# Demand Constraint

def total_demand(m, i, j):
    return (m.x[i, j] * 40 * m.sa[i, j] + m.y[i, j] == m.demand[i, j])


model2.total_demand = Constraint(model2.i, model2.j, rule=total_demand)

# Outsource Demand Constraints
model2.outsource_demand = ConstraintList()
for j in model2.j:
    for i in model2.i:
        if i == 'A':
            model2.outsource_demand.add(expr = model2.y[i,j] <= 0.3*model2.demand[i,j])
        elif i == 'B':
            model2.outsource_demand.add(expr = model2.y[i,j] <= 0.4*model2.demand[i,j])


In [ ]:
# Objective function
model2.value = Objective(expr = sum(sum(model2.x[i,j]*FTE_Salary[i,j]+ model2.y[i,j]*UnitOutSourceCost[i,j] for i in model2.i) for j in model2.j),sense= minimize)

In [ ]:
# Invoking the solver
result = SolverFactory('glpk').solve(model2)
result.write()

In [ ]:
# Print the value of the objective function
optimized_cost_best = round(model2.value()/1000000,2)

print("\n\n The company need to spend around ${}m in total for the application approval process" .format(optimized_cost_best))


**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
model2.pprint

In [ ]:
# Creating dataframe for the results
Output_best = []

for i in model2.i:
    for j in model2.j:
        # cost for the application approval process
        no_of_fte = model2.x[i, j].value
        demand = model2.demand[i, j]
        no_of_outsource_app = model2.y[i, j].value
        no_of_FTE_processed_app = model2.x[i,j].value*40*model2.sa[i,j]
        cost = model2.x[i, j].value*FTE_Salary[i, j] + model2.y[i, j].value*UnitOutSourceCost[i, j]

        percent_outsourced_app = round((no_of_outsource_app/demand)*100,1)
        cost_per_app = round(cost/demand,1)

        Output_best.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,cost_per_app])
print(Output_best)

In [ ]:
Output_best = pd.DataFrame(Output_best, columns = ['State', 'Month', 'No of FTE best',"Demand",'No of outsource app best',"No of FTE processed App best" , 'Estimated Cost best', 'percent outsourced app best',  'cost per application best'])

Output_best[['Estimated Cost in mn $ best']] = Output_best[['Estimated Cost best']]/1000000
Output_best['Demand'] = Output_best['Demand'].astype(int)
Output_best['No of outsource app best'] = Output_best['No of outsource app best'].astype(int)
Output_best['No of FTE processed App best'] = Output_best['No of FTE processed App best'].astype(int)
Output_best = Output_best.round({"No of FTE best":1, "No of outsource app best":0, "Estimated Cost in mn $ best":1})
#Output_best[['Cost per application best']] = Output_best['Estimated Cost best']/Output_best['Demand']
# Print the Output values
print("\n\n" ,Output_best)


In [ ]:
total_outsourcing =  sum(Output_best['No of outsource app best'])
total_cost = sum(Output_best['Estimated Cost best'])
optimal_staff_members = round(Output_best['No of FTE best'].mean(),1)

total_demand =  sum(Outpu_best['Demand'])
average_cost_per_appln = round(total_cost/total_demand,2)
percent_outsourced_app_total = round((total_outsourcing/total_demand)*100,1)
print("Total Outsourced Application: ", total_outsourcing)
print("Total Demand Application: ", total_demand)
print("Optimal Staff Members : ", optimal_staff_members)

#### 3.2.2 Percentage of outsourced applications

In [ ]:
# write your code here
print("Total Outsourced Application % is : ", percent_outsourced_app_total)


#### 3.2.3 Average cost per application

In [ ]:
# write your code here
print("Average Cost per Application: ", average_cost_per_appln)


In [ ]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_best'
Output_best.to_excel(writer, sheet_name='Output_best')


# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

In [1]:
import jovian

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# df = Output_actual[['Month', 'Demand', 'No of outsource app',"No of FTE processed App"]].groupby(['Month']).sum()
# df[["Percentage of outsource app"]] = df['No of outsource app']/df['Demand']*100
# df[['Percentage of FTE processed app']] = df['No of FTE processed App']/df['Demand']*100
# ax = df[['Percentage of outsource app','Percentage of FTE processed app']].plot(kind='bar', stacked=True, figsize=(10,10))
# ax_set_ylabel("Percentage", fontsize=12)
# plt.title("Stacked bar chart between percentages of FTE Processed app and Outsourced app", fontsize=12)
# plt.show()

In [ ]:
# df_new = pd.concat([Output_actual[['Cost per application actual','Month']], Output_worst[['Cost per application worst']], Output_best[['Cost per application best']]], axis=1)
# df_new = df_new.groupby(['Month']).mean()
# df_new.plot.line(figsize=[10,5])
# plt.xlabel("Month", figsize=10)
# plt.ylabel("Cost", figsize=10)
# plt.title()
# plt.show()
